In [1]:
import json

import pandas as pd
from exp_utility import get_adapted_curr_layer_class_dist, get_all_defined_classes_per_layer, get_layer_class_dist, map_all_classes_to_merged_list, remove_empty_lists


### 1. Merge all batches & get top_1 classe for all captions per layers
#### 1.1 Not all generated answers by LLM match the json dictionary format that is required and if so they are removed. This results in around 1.3% loss in data

In [9]:
no_layers = 24
no_batch = 1
samples_per_batch = 500
dataset_size = no_batch * samples_per_batch 

top1 = {}
for l in range(no_layers):
    layer_data = []
    for b in range(no_batch): #FIXME change the path and give batch size and layer no 
        with open(f'../data/interpret/label_top3_llama70B/classification_llama_70B_top3_labels_layer_{l}_caps_batch{b}.json') as f:
            data = json.load(f)
        layer_data.extend(data)
    curr,l,e = get_all_defined_classes_per_layer(layer_data,l,dataset_size)
    print(f'retrieved top1 classes for {len(curr)} at layer {l+1}')
    top1[f'layer_{l}'] = curr

retrieved top1 classes for 441 at layer 1
retrieved top1 classes for 481 at layer 2
retrieved top1 classes for 477 at layer 3
retrieved top1 classes for 479 at layer 4
retrieved top1 classes for 494 at layer 5
retrieved top1 classes for 491 at layer 6
retrieved top1 classes for 496 at layer 7
retrieved top1 classes for 497 at layer 8
retrieved top1 classes for 500 at layer 9
retrieved top1 classes for 500 at layer 10
retrieved top1 classes for 499 at layer 11
retrieved top1 classes for 488 at layer 12
retrieved top1 classes for 496 at layer 13
retrieved top1 classes for 498 at layer 14
retrieved top1 classes for 499 at layer 15
retrieved top1 classes for 499 at layer 16
retrieved top1 classes for 499 at layer 17
retrieved top1 classes for 500 at layer 18
retrieved top1 classes for 500 at layer 19
retrieved top1 classes for 500 at layer 20
retrieved top1 classes for 500 at layer 21
retrieved top1 classes for 498 at layer 22
retrieved top1 classes for 499 at layer 23
retrieved top1 class

### 2. Process the top classes and store in panda dataframe
#### 2.1 Calculate the distribution of each class per layer (this step requires some postprocessing and cleaning using the provided helper functions)
#### 2.2 Generate dataframe with percentage of each recognized class in the columns and layers as rows

In [10]:
all_labels, list_classes = map_all_classes_to_merged_list()

df_list = []
for layer_no in range(24):
    curr_layer_class_dist, c = get_adapted_curr_layer_class_dist(top1,layer_no,all_labels)
    curr_layer_class_dist = remove_empty_lists(curr_layer_class_dist)
    df = get_layer_class_dist(curr_layer_class_dist, list_classes)
   
    
    df_list.append(df.rename(columns={'Percentage':layer_no+1}).pivot_table(columns='Class'))

In [11]:
classification_data = pd.concat(df_list)
classification_data.to_csv("../data/CSV_files/classification_data_500.csv", index=False)